In [ ]:
import pandas as pd
import numpy as np
import datetime as date
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv("sap_storing_data_hu_project.csv")

__1.1 Target variabele bepalen__

De target variabele waarnaar we willen gaan voorspellen wordt de eindtijd van de storing. 

__1.2 Inhoud en betekenis target variabele__


We willen voor elk ongeval voorspellen hoe lang de storing gaat duren, hiervoor willen we de eindtijd van de storing voorspellen. Om te kijken naar hoe snel eerdere problemen opgelost zijn hebben we een nieuwe variabele nodig: namelijk de tijd tussen het tijdstip van de melding van een ongeval en het tijdstip waarop het de storing verholpen is. 

In [ ]:
df[['stm_sap_meld_ddt', 'stm_sap_storeind_ddt']]

__1.3 Target variabele geschikt maken voor model__

In [ ]:
df['stm_sap_meld_ddt'] = pd.to_datetime(df['stm_sap_meld_ddt'], format='%d/%m/%Y %H:%M:%S')
df['stm_sap_storeind_ddt'] = pd.to_datetime(df['stm_sap_storeind_ddt'], format='%d/%m/%Y %H:%M:%S')

In [ ]:
# df['diff'] = datetime.strptime(df['stm_sap_meld_ddt'], FMT) - datetime.strptime(df['stm_sap_storeindtijd'], FMT).df['fh']
df['diff_meld_strend'] = df['stm_sap_storeind_ddt']-df['stm_sap_meld_ddt']

In [ ]:
df[['diff_meld_strend','stm_sap_meld_ddt', 'stm_sap_storeind_ddt']]

Nu tussen andere waardes, de tijd van melden en de tijd van functie herstel.

In [ ]:
# df['stm_sap_meldtijd'] = pd.to_datetime(df['stm_sap_meldtijd'], format='%H:%M:%S', errors='coerce')
# df['stm_fh_tijd'] = pd.to_datetime(df['stm_fh_tijd'], format='%H:%M:%S', errors='coerce')

In [ ]:
# df['diff_meld_fhtijd'] = (df['stm_fh_tijd'] - df['stm_sap_meldtijd'] )
# df['diff_meld_fhtijd2'] = (df['stm_sap_meldtijd'] - df['stm_fh_tijd'])

Data dat iets is opgelost in de toekomst is onzin dus die verweideren.

In [ ]:
df[df['stm_sap_storeind_ddt'] >= date.datetime.today()][['stm_sap_storeind_ddt','stm_sap_meld_ddt']]
df.drop(df[df['stm_sap_storeind_ddt'] >= date.datetime.today()][['stm_sap_storeind_ddt','stm_sap_meld_ddt']].index,inplace=True)

Een verschil van 10 jaar slaat ook nergens op.<br>
En de tijd van ernstige reparaties met spoen is altijd binnen dezelfde dag nog gedaan dus alles wat langer dan een dag duurt gaat er uit.

In [ ]:
cols = ['stm_sap_meld_ddt','stm_sap_storeind_ddt','diff_meld_strend']
df.drop(df[df.apply(lambda x: x['diff_meld_strend'].days >= 1, axis=1)][cols].index, inplace = True)

Rijen waar het probleem eerder is opgelost dan de melding is ook onbetrouwbare data.

In [ ]:
df.drop(df[df['diff_meld_strend'] < df['diff_meld_strend'][0]][cols].index, inplace = True)

In [ ]:
# neg = df[df['diff_meld_strend'] < df['diff_meld_strend'][0]][cols]
# print(neg.shape)# 262 meldingen NA dat de storing was opgelost
# display(neg)
# display(neg.describe())

In [ ]:
pos = df[df['diff_meld_strend'] > df['diff_meld_strend'][0]][cols]
print(pos.shape)# 659813 meldingen NA dat de storing was opgelost
display(pos)
display(pos.describe())

In [ ]:
box_all = plt.boxplot(df['diff_meld_strend'],labels=['Spreiding van melding tot storing einde'])

In [ ]:
box_pos = plt.boxplot(pos['diff_meld_strend'],labels=['Spreiding van melding tot storing einde'])

In [ ]:
box_neg = plt.boxplot(neg['diff_meld_strend'],labels=['Spreiding van melding tot storing einde'])

Voor een baseline model kiezen we de meest voorkomende tijd.<br>
Dat is het getal dat je onder de mediaan vind: `02:17:00`.
<br><br>
Als baseline zeggen we dan ook: "Alle storingen zijn binnen 2 uur en 17 minuten opgelost.".

In [ ]:
amntInTime = df[df['diff_meld_strend'] < date.timedelta(hours=2,minutes=17)][cols].count()[0]
total = df.shape[0]
print(f'{amntInTime / total * 100}% van de tijd heeft het baseline model het goed.')